# Equilibirum notes

In [ ]:
using Revise
using FUSE
using Plots;
FUSE.logging(Logging.Info);

### Comparing Solovev equilibirum with CHEASE equilibrium to the original ITER ods equilibrium

In [ ]:
### Solovev init equilibrium
ini, act = FUSE.case_parameters(:ITER, init_from=:ods);
ini.equilibrium.B0 = 5.3
dd = FUSE.init(ini,act)
eq_plot = plot(dd.equilibrium, label="ITER ods equilibrium")

### CHEASE equilibrium
act.ActorEquilibrium.model = :CHEASE
act.ActorCHEASE.free_boundary = false
FUSE.ActorEquilibrium(dd, act)
display(plot!(eq_plot, dd.equilibrium, label="CHEASE equilibrium"))
FUSE.ActorEquilibrium(dd,act)
display(plot!(eq_plot,dd.equilibrium, label="CHEASE equilibrium 3"))
FUSE.ActorEquilibrium(dd,act)
display(plot!(eq_plot,dd.equilibrium, label="CHEASE equilibrium 4"))

In [ ]:
FUSE.digest(dd)

### Finding a MXH parametrization of the boundary for ITER

In [ ]:
import MillerExtendedHarmonic: flat_coeffs
#dd, ini, act = FUSE.init(:ITER, init_from=:ods); start with CHEASE boundary from above
eqt = dd.equilibrium.time_slice[]
eqt_bound = eqt.boundary.outline
psi_close = (eqt.profiles_1d.psi[end] + 5 * eqt.profiles_1d.psi[end]) / 6
r_bound, z_bound = IMAS.flux_surface(eqt, psi_close)

eq_plot = plot(r_bound, z_bound, label="1.0 of original boundary shape")
for order in [4]
    mxh = IMAS.MXH(r_bound, z_bound, order)
    @show round.(flat_coeffs(mxh), digits=5)
    plot!(mxh, label="MXH $order")
end
display(plot!(eqt_bound.r, eqt_bound.z, label="original boundary shape", color="black", marker="x", lw=0, markersize=1))

### Comparing CHEASE deadstart from MXH parametrization vs. Miller parametrization

In [ ]:
ini, act = FUSE.case_parameters(:ITER, init_from=:scalars);
ini

In [ ]:
# Solovev deadstart (default)
dd, ini, act = FUSE.init(:ITER, init_from=:ods);
ods_plot = plot(dd.equilibrium, label="ITER ODS eq")
display(ods_plot)

# CHEASE deadstart MXH
dd = IMAS.dd()
ini, act = FUSE.case_parameters(:ITER, init_from=:scalars);
ini.equilibrium.boundary_from = :MXH_params
act.ActorEquilibrium.model = :CHEASE
dd = FUSE.init(dd, ini, act)
mxh_plot = plot(dd.equilibrium, label="CHEASE deadstart MXH")
display(mxh_plot)

# CHEASE deadstart MILLER
dd = IMAS.dd()
ini, act = FUSE.case_parameters(:ITER, init_from=:scalars);
ini.equilibrium.boundary_from = :scalars
act.ActorEquilibrium.model = :CHEASE
act.ActorCHEASE.free_boundary = false
dd = FUSE.init(dd, ini, act)
miller_plot = plot(dd.equilibrium, label="CHEASE deadstart Miller")
display(miller_plot)

### Make new FPP boundary from scratch

In [ ]:
# CHEASE deadstart Miller
dd = IMAS.dd()
ini, act = FUSE.case_parameters(:FPP, version=:v1_demount, init_from=:scalars);
ini.equilibrium.boundary_from = :scalars
act.ActorEquilibrium.model = :CHEASE
act.ActorCHEASE.free_boundary = true

ini.equilibrium.κ = 2.2
ini.equilibrium.ζ = 0.0

dd = FUSE.init(dd, ini, act)
mxh_plot = plot(dd.equilibrium, label="CHEASE deadstart Miller")
display(mxh_plot)

FUSE.digest(dd)